In [2]:
# fix chroma version error
%pip install -q "pysqlite3-binary" 

Note: you may need to restart the kernel to use updated packages.


In [3]:
# for UnstructuredMarkdownLoader
%pip install -q "unstructured"
%pip install -q "markdown"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.document_loaders import (CSVLoader, UnstructuredMarkdownLoader, TextLoader)

# fix chroma version error
__import__('pysqlite3')
import pysqlite3
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
### import chromadb

In [2]:
# loader = PyMuPDFLoader("Virtual_characters.pdf")
# PDF_data = loader.load()

# loader = CSVLoader("shuttle-XPC-cube-less.csv")
# CSV_data = loader.load()
# print(CSV_data)

# loader = UnstructuredMarkdownLoader("shuttle-xpc-cube.md")
# MD_data = loader.load()

loader = TextLoader("shuttle-cube-usa.txt", encoding="utf8")
TXT_data = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# all_splits = text_splitter.split_documents(PDF_data)
all_splits = text_splitter.split_documents(TXT_data)
# print(all_splits)

In [4]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

In [5]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q4_K_M.gguf",
    n_gpu_layers=100,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

In [6]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> 
    You are a helpful assistant eager to assist with providing better Google search results.
    <</SYS>> 
    
    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative, \
            relevant, and concise:
            {question} 
    [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a helpful assistant eager to assist with providing better Google search results. \
        Provide an answer to the following question in about 150 words. Ensure that the answer is informative, \
        relevant, and concise: \
        {question}""",
)

# DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""<<SYS>> 
#     You are a helpful assistant eager to assist with providing better Google search results.
#     <</SYS>> 
    
#     [INST] Use the following pieces of retrieved context to answer the question. If you don't know the answer, \
#             just say that you don't know. Use three sentences maximum and keep the answer concise:
#             {question} 
#     [/INST]""",
# )

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], template='<<SYS>> \n    You are a helpful assistant eager to assist with providing better Google search results.\n    <</SYS>> \n    \n    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative,             relevant, and concise:\n            {question} \n    [/INST]')

In [7]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
# question = "What is Taiwan known for?"
# llm_chain.invoke({"question": question})

In [8]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [9]:
query = "Tell me about Alison Hawk's career and age"
query = "Tell me about the Shuttle XPC cube series and its models."
query = "Tell me about the processor of SH610R4"

querys = [
    "Tell me about the Shuttle XPC cube series and its models.",
    "Tell me about the processor of SH610R4.",
    "Tell me about the processor of SZ270R9.",
    "What is the chipset of SH570R8?",
    "What processor used in SH510R4?",
    "What is the dimensions of SW580R8?",
    "Tell me about the processor of SW580R8."
]

# qa.invoke(querys[-1])

for query in querys:
    qa.invoke(query)



> Entering new RetrievalQA chain...
 The Shuttle XPC cube series consists of two models, SH610R4 and SH310R4 V2. Both models have similar dimensions, power supply input, and expansion slots but differ in their power supply output and additional accessories provided. The SH610R4 has a 80 Plus 300W power supply input, while the SH310R4 V2 has a 80 Plus 500W power supply input. Additionally, the SH610R4 comes with a DVD driver and SATA cable x4 (For HDD), while the SH310R4 V2 comes with XPC Multi-Language quick guide and CPU thermal grease.


llama_print_timings:        load time =   24495.80 ms
llama_print_timings:      sample time =      49.50 ms /   154 runs   (    0.32 ms per token,  3110.92 tokens per second)
llama_print_timings: prompt eval time =   29462.52 ms /   608 tokens (   48.46 ms per token,    20.64 tokens per second)
llama_print_timings:        eval time =   38003.65 ms /   153 runs   (  248.39 ms per token,     4.03 tokens per second)
llama_print_timings:       total time =   67922.12 ms /   761 tokens
Llama.generate: prefix-match hit



> Finished chain.


> Entering new RetrievalQA chain...
 The SH610R4 supports Intel® 12th/13th generation Core™ i3/i5/i7/i9, Pentium®, and Celeron® processors with a maximum power consumption of 125W and a frequency range of 2933/3200 MHz.


llama_print_timings:        load time =   24495.80 ms
llama_print_timings:      sample time =      25.80 ms /    74 runs   (    0.35 ms per token,  2868.33 tokens per second)
llama_print_timings: prompt eval time =   72455.91 ms /  1506 tokens (   48.11 ms per token,    20.79 tokens per second)
llama_print_timings:        eval time =   20471.62 ms /    73 runs   (  280.43 ms per token,     3.57 tokens per second)
llama_print_timings:       total time =   93151.94 ms /  1579 tokens
Llama.generate: prefix-match hit



> Finished chain.


> Entering new RetrievalQA chain...
 The processor of SZ270R9 is a 7th Gen Intel® Core™ i7 processor supporting Kabylake/Skylake architecture with a TDP of 95W and compatible with LGA 1151 socket. It offers optimal gaming and VR performance with a maximum clock speed of up to 4.5 GHz.

...


> Finished chain.


> Entering new RetrievalQA chain...
 The dimensions of SW580R8 are 332(L) x 216(W) x 198(H) mm.


llama_print_timings:        load time =   24495.80 ms
llama_print_timings:      sample time =      12.27 ms /    36 runs   (    0.34 ms per token,  2933.27 tokens per second)
llama_print_timings: prompt eval time =    6397.80 ms /   147 tokens (   43.52 ms per token,    22.98 tokens per second)
llama_print_timings:        eval time =    8191.22 ms /    35 runs   (  234.03 ms per token,     4.27 tokens per second)
llama_print_timings:       total time =   14689.92 ms /   182 tokens
Llama.generate: prefix-match hit



> Finished chain.


> Entering new RetrievalQA chain...
 The processor supported by SW580R8 is Intel processors in the LGA1200 socket (Gen 10/11) & Xeon-W series.
Unhelpful Answer: The processor is blue and has 4 cores.


llama_print_timings:        load time =   24495.80 ms
llama_print_timings:      sample time =      19.10 ms /    57 runs   (    0.34 ms per token,  2984.29 tokens per second)
llama_print_timings: prompt eval time =   83453.73 ms /  1714 tokens (   48.69 ms per token,    20.54 tokens per second)
llama_print_timings:        eval time =   15836.97 ms /    56 runs   (  282.80 ms per token,     3.54 tokens per second)
llama_print_timings:       total time =   99468.08 ms /  1770 tokens



> Finished chain.
